# Julia version of bonding models for PES !!

### Basic Optimization tests

In [113]:
using Optim, LsqFit # opt libs
using DelimitedFiles, MLUtils, BenchmarkTools # utils

In [115]:
# standard opt:
rosenbrock(x) =  (1.0 - x[1])^2 + 99.0 * (x[2] - x[1]^2)^2
result = optimize(rosenbrock, [1.1, 2.3], BFGS())

 * Status: success

 * Candidate solution
    Final objective value:     5.264688e-17

 * Found with
    Algorithm:     BFGS

 * Convergence measures
    |x - x'|               = 9.47e-08 ≰ 0.0e+00
    |x - x'|/|x'|          = 9.47e-08 ≰ 0.0e+00
    |f(x) - f(x')|         = 3.53e-15 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 6.71e+01 ≰ 0.0e+00
    |g(x)|                 = 3.92e-10 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    15
    f(x) calls:    46
    ∇f(x) calls:   46


In [116]:
# examples of "closures", a technique to fix args!
f(x::Vector, a, b) = (x[1] - a)^2 + (x[2] - b)^2

using Optim
g(x::Vector) = f(x, 3, 4)
optimize(g, [0.,0.])

 * Status: success

 * Candidate solution
    Final objective value:     1.226133e-10

 * Found with
    Algorithm:     Nelder-Mead

 * Convergence measures
    √(Σ(yᵢ-ȳ)²)/n ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    50
    f(x) calls:    100


In [117]:
# another closure example in Julia, anonymous function:
optimize(x -> f(x,3,4), [0.,0.]) 

 * Status: success

 * Candidate solution
    Final objective value:     1.226133e-10

 * Found with
    Algorithm:     Nelder-Mead

 * Convergence measures
    √(Σ(yᵢ-ȳ)²)/n ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    50
    f(x) calls:    100


In [211]:
# functional form:
function f_ratpot_2(Θ, R, M)
    #=
    ansatz 1 for diatomic potential
    params:
        - Θ := training parameters, vector ()
        - R := distances, vector
    =#
    # unroll coefficients
    a = Θ[1:M]
    b = Θ[M+1:2*M]
    c = Θ[2*M+1:3*M+4]
    d = Θ[3*M+5:4*M+7]
    
    # b_i ≥ 0 for i > 1:
    t = b[2:M]
    bool = t .< 0.
    t[bool] = -t[bool]
    b[2:M] = t
    
    # d_i ≥ 0 for i > 0:
    bool = d .< 0.
    d[bool] = -d[bool]    
    println(b)
    println(d)
    
    # evaluate P:
    P = c[end-1]
    P = P .* ((R .- a[1]).^2 .+ (b[1] .* R))
    for i=2:M
        P .*= (R .- a[i]).^2 .+ (b[i]*R)
    end

    
    # eval Q:
    Q = (R .+ d[end]).*R
    for i=1:M+2
        Q .*= (R .- c[i]).^2 .+ d[i].*R
    end
    
    # eval potential:
    V = c[end] .+ (P ./ Q)
    return V
end

f_RMSE(X, Y) = √(sum((X .- Y) .^ 2)/length(X))

function f_least_squares(f_eval, X, Y, f_args...)
    Y_pred = f_eval(X, f_args...)
    return sum((Y .- Y_pred).^2)
end

f_least_squares (generic function with 2 methods)

In [212]:
M = 2
Θ = collect(LinRange(-1., -2., 21))
Θ = Θ[1:4*M+7] # 4M+7 params
println(Θ, size(Θ))
R = collect(LinRange(0., 5., 6)) .+ 1
println(R)
f_ratpot_2(Θ, R, M)

[-1.0, -1.05, -1.1, -1.15, -1.2000000000000002, -1.25, -1.2999999999999998, -1.35, -1.4, -1.4500000000000002, -1.5, -1.55, -1.6, -1.65, -1.7](15,)
[1.0, 2.0, 3.0, 4.0, 5.0, 6.0]
[-1.1, 1.15]
[1.5, 1.55, 1.6, 1.65, 1.7]


6-element Vector{Float64}:
 -1.4538848497222299
 -1.4504032610414908
 -1.450089540167252
 -1.4500280979713995
 -1.450010836561844
 -1.4500048065726545

#### Optimize ratpot 2 !!

In [66]:
# prepare data:
H_data = readdlm("data/h2/h2_ground_w.txt")
R = H_data[:, 1]; V = H_data[:, 2]
Xs, Ys = shuffleobs((R, V))
train_data, test_data = splitobs((Xs, Ys); at=0.8)
R_train = copy(train_data[1]); V_train = copy(train_data[2]);
length(R_train)

536

In [106]:
f(x, y=1, z=2) = x + y + z
function g(fx, a, f_args...)
    return a*f(f_args...) 
end


g(f, 2, 1, 1, 3)

10

In [210]:
function fmin(x,a,b,c)
   return sum(x.^2) + a*b + c
end

a0 = 3.
b0 = 2.
c0 = 4.
res = optimize(x -> fmin(x,a0,b0,c0), [100., 100.], LBFGS(); autodiff = :forward)
println(res)
println(res.minimizer, res.minimum)
#println(f(res.minimizer, a0, b0))

 * Status: success

 * Candidate solution
    Final objective value:     1.000000e+01

 * Found with
    Algorithm:     L-BFGS

 * Convergence measures
    |x - x'|               = 1.00e+02 ≰ 0.0e+00
    |x - x'|/|x'|          = Inf ≰ 0.0e+00
    |f(x) - f(x')|         = 2.00e+04 ≰ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 2.00e+03 ≰ 0.0e+00
    |g(x)|                 = 0.00e+00 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    1
    f(x) calls:    3
    ∇f(x) calls:   3

[0.0, 0.0]10.0


### 8.1 Bonding features

In [75]:
function t_R_fun(R, R_up, R_low, e)
    R2 = R.^2
    return ((R2 .- R_low^2)./(R_up^2 .- R2)).^e
end

#function s_bond_strength

t_R_fun (generic function with 1 method)

In [76]:
R = [2.5, 3.1, 5, 1.1]
@benchmark t_R_fun(R, 6, 1, 2)

BenchmarkTools.Trial: 10000 samples with 982 evaluations.
 Range (min … max):  59.369 ns …  1.314 μs  ┊ GC (min … max): 0.00% … 92.58%
 Time  (median):     61.507 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   67.030 ns ± 56.162 ns  ┊ GC (mean ± σ):  5.19% ±  5.87%

  ▄██▄▅▄▃▂▁▁                                                  ▂
  ███████████▇▇▇▆▆▆▅▄▅▆▄▄▄▆▄▃▅▄▁▃▁▄▁▁▃▄▃▃▁▁▁▁▁▃▁▁▃▁▁▃▅▆▇▇██▇▇ █
  59.4 ns      Histogram: log(frequency) by time       119 ns <

 Memory estimate: 192 bytes, allocs estimate: 2.